# Checking for convergence in dE with cut off radius
This script checks for the convergence in the change in lattice energy, dE, before and after performing a Monte Carlo move in Eris. The lattice energy is calculated by performing a summation over lattice electrostatics out to a finite cut off radius. dE is used in Eris to evolve the system towards, i.e. the accept/ reject criteria for Monte Carlo moves (nearest-neighbour Cu-Zn swaps) is based on this value.

This test is used to determine a suitable value to set `ElectroStaticsCutOff` to in eris.cfg for subsequent simulations.

## Eris simulation flags
Set the flag `ElectrostaticsCheck` in eris.cfg to be true. This will then output files dE_conv_T_TEMP.dat, where the file appends for each different attempted move.

### Step 1 - Split up the attempted MC moves
Use the bash script below to separate the outputted dE calculated for different MC moves. This script below generates a file where all odd columns are the cut off radius (so are the same) and each even column corresponds to the dE calculated for a different MC move with increasing ut off radius.

In the bash script below, enter the temperature range and step size used in the Eris simulation and also add the value for loop_step generated in the python script below based on lattice dimensions you enter into the script.

In [ ]:
import math

# User inputs
X_dim=28
Y_dim=28
Z_dim=28

# Define half smallest dim of lattice (used as r_cutoff,max for dE check)
# Add 1 to value for r_cutoff,max to account for top comment line
loop_step=math.floor(min(X_dim,Y_dim,Z_dim)/2)+1
print(loop_step)

15.0


In [ ]:
%%bash

# User inputs
loop_step=15 # Please take value outputted above (as an int)
T_min=0
T_max=1000
T_step=50

# -------------------------------------------------------------------------------
loop_max=100

# Move dE convergence outputs to a separate directory, keep things tidy
mkdir data
mv dE_conv* data

for temp in $(seq $T_min $T_step $T_max)
do
    temp_formatted=$(printf "%04d" $temp)

    # Loop over dE input file, write each segment to a temporary file, paste together into columns of one file, delete tmp files
    for i in $(seq 1 $loop_step $loop_max)
    do
        # Copy next segment to new file
        loop_end=$(echo "$i+$loop_step-1" | bc)
        sed -n "$i","$loop_end"p data/dE_conv_check_T_${temp_formatted}.dat > data/tmp_${i}.dat 

    done  

# Paste all temporary files together, 2 columns for each different move attempted for dE calculation
paste data/tmp* > data/dE_split_cols_T_${temp_formatted}.dat
# Delete temporary files
rm data/tmp*

done

### Step 2 - Plotting!
The next script plots dE calculated for one attempted MC move at each simulation temperature calculation with a cut off radius of 1 to r_max for the summation of lattice electrostatics.

The first column of dE_split_cols_T_TEMP.dat is always used for r in the plot (column index 0), then all odd number columns corresponds to the dE calculated for different attempted MC moves. Not that we did not distinguish between accepted and reject moves here.

Choose the temperatures you wish to generate the plot for. Currently this is just set to a list of 6 temperatures.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.pylabtools import figsize
figsize(14, 12)

# Set list of temperatures
T=(50, 100, 300, 500, 700, 900)

# --------------------------------------------------------------------------

data_file0='data/dE_split_cols_T_'+str(T[0]).zfill(4)+'.dat'
data0= np.genfromtxt(data_file0, delimiter = ' ')
data_file1='data/dE_split_cols_T_'+str(T[1]).zfill(4)+'.dat'
data1= np.genfromtxt(data_file1, delimiter = ' ')
data_file2='data/dE_split_cols_T_'+str(T[2]).zfill(4)+'.dat'
data2= np.genfromtxt(data_file2, delimiter = ' ')
data_file3='data/dE_split_cols_T_'+str(T[3]).zfill(4)+'.dat'
data3= np.genfromtxt(data_file3, delimiter = ' ')
data_file4='data/dE_split_cols_T_'+str(T[4]).zfill(4)+'.dat'
data4= np.genfromtxt(data_file4, delimiter = ' ')
data_file5='data/dE_split_cols_T_'+str(T[5]).zfill(4)+'.dat'
data5= np.genfromtxt(data_file5, delimiter = ' ')
    
# It is here that the second column of data to plot should be changed (this is the column scaled by 0.025)
# Different odd column numbers should be trialed until a sensible MC move has been attempted
# Multiply each dE by 0.025 (kBT at 300K)
plt.plot(data0[:,0], 0.025*data0[:,1], label='T='+str(T[0])+'K')
plt.plot(data1[:,0], 0.025*data1[:,5], label='T='+str(T[1])+'K')
plt.plot(data2[:,0], 0.025*data2[:,5], label='T='+str(T[2])+'K')
plt.plot(data3[:,0], 0.025*data3[:,1], label='T='+str(T[3])+'K')
plt.plot(data4[:,0], 0.025*data4[:,9], label='T='+str(T[4])+'K')
plt.plot(data5[:,0], 0.025*data5[:,5], label='T='+str(T[5])+'K')

plt.xlabel('r (lattice units)')
plt.ylabel('dE (eV)')
plt.legend()
plt.show()